In [2]:
# Import libraries
import pandas as pd
import numpy as np
import category_encoders as ce
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.ensemble import RandomForestRegressor


# Passenger volume prediction model

## Importing and checking dataset

In [5]:
# Import dataset
df_flights = pd.read_csv("../data/Filghts TEC_Valid.csv")

In [6]:
# Check dataset
df_flights

,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings
0,ab954014077430bd842cfa305a55c0f8,XA-VBY,AT,AZ,Ciudad Fronteriza,Ciudad Principal,2023-10-19 11:40:00,2023-10-19 14:25:00,240,229.0,157.0
1,efd86c996035dacdca7a0ccb2560dda1,XA-VIX,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,2023-07-03 00:55:00,2023-07-03 04:55:00,186,197.0,109.0
2,6cfa1bbaa44f08fc7d3061f034a6a5ce,XA-VBV,AW,AS,MX Amigos y Familia,Ciudad Principal,2024-02-16 17:10:00,2024-02-16 17:55:00,220,NaN,NaN
3,dd0fad3248951d2f71d63e6279aeaa4b,XA-VBW,AW,AS,MX Amigos y Familia,Ciudad Principal,2023-06-26 15:15:00,2023-06-26 15:55:00,220,200.0,142.0
4,d0987ee648eea254063bfe2b39571b67,XA-VAP,BA,AB,Playa,Ciudad Principal,2023-02-10 08:40:00,2023-02-10 09:50:00,186,162.0,90.0
...,...,...,...,...,...,...,...,...,...,...,...
245748,cc7c1c5e6fd132fd0bdab3a35aac33c0,XA-VBK,BM,BT,Playa,Ciudad Fronteriza,2023-12-29 07:30:00,2023-12-29 14:50:00,240,189.0,86.0
245749,ef32da2731db80faa8b9f5030979a016,9H-MLV,AW,BT,Playa,Ciudad Principal,2024-04-20 10:45:00,2024-04-20 14:00:00,178,NaN,NaN
245750,9c8970e9836d9c5ef9415bfa93c3f408,XA-VAC,AT,BT,Playa,Ciudad Principal,2023-12-22 12:50:00,2023-12-22 16:10:00,180,163.0,95.0
245751,3c15779202b13158f78e8a8afe377250,NaN,AW,BT,Playa,Ciudad Principal,2024-10-04 10:00:00,2024-10-04 13:15:00,240,NaN,NaN


We can check there are 245,753 rows, but we need to check if all rows are unique flights

In [7]:
#Check unique flights
len(df_flights["Flight_ID"].unique())

238055

There are 238,055 unique flights id, this is because some flights have more than one destination

In [8]:
# Check Null Values in dataset
nulls_flights = df_flights.isnull().sum()


# Review basic statistics for outliers or unusual values
stats_flights = df_flights.describe(include='all')


nulls_flights, stats_flights

(Flight_ID                0
 Aeronave             80390
 DepartureStation         1
 ArrivalStation           1
 Destination_Type         1
 Origin_Type              1
 STD                      0
 STA                      0
 Capacity                 0
 Passengers          123525
 Bookings            123525
 dtype: int64,
                                Flight_ID Aeronave DepartureStation  \
 count                             245753   165363           245752   
 unique                            238055      104               41   
 top     d2093576a84c20bb81f426f1596a0e5c   XA-VBU               AW   
 freq                                   4     2753            43082   
 mean                                 NaN      NaN              NaN   
 std                                  NaN      NaN              NaN   
 min                                  NaN      NaN              NaN   
 25%                                  NaN      NaN              NaN   
 50%                                  

There are a lot of NaN in Aeronave, we are not going to do anything about it because this column isn't important to predict passengers, since passengers don´t choose flights depending of the aircraft.

Passengers and booking has many null values ​​because since January 1, 2024 it does not contain that information.

In [9]:
# Separate the STD AND STA column into year, month, day and time. Rounding the hour column

# Convert STD and STA columns to datetime
df_flights['STD'] = pd.to_datetime(df_flights['STD'])
df_flights['STA'] = pd.to_datetime(df_flights['STA'])

# Extract year, month, day and time from STD
df_flights['STD_Year'] = df_flights['STD'].dt.year
df_flights['STD_Month'] = df_flights['STD'].dt.month
df_flights['STD_Day'] = df_flights['STD'].dt.day
df_flights['STD_Hour'] = df_flights['STD'].dt.hour + round(df_flights['STD'].dt.minute / 60)

# Extract year, month, day and time from STA
df_flights['STA_Year'] = df_flights['STA'].dt.year
df_flights['STA_Month'] = df_flights['STA'].dt.month
df_flights['STA_Day'] = df_flights['STA'].dt.day
df_flights['STA_Hour'] = df_flights['STA'].dt.hour + round(df_flights['STA'].dt.minute / 60)

In [10]:
df_flights.head()

,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings,STD_Year,STD_Month,STD_Day,STD_Hour,STA_Year,STA_Month,STA_Day,STA_Hour
0,ab954014077430bd842cfa305a55c0f8,XA-VBY,AT,AZ,Ciudad Fronteriza,Ciudad Principal,2023-10-19 11:40:00,2023-10-19 14:25:00,240,229.0,157.0,2023,10,19,12.0,2023,10,19,14.0
1,efd86c996035dacdca7a0ccb2560dda1,XA-VIX,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,2023-07-03 00:55:00,2023-07-03 04:55:00,186,197.0,109.0,2023,7,3,1.0,2023,7,3,5.0
2,6cfa1bbaa44f08fc7d3061f034a6a5ce,XA-VBV,AW,AS,MX Amigos y Familia,Ciudad Principal,2024-02-16 17:10:00,2024-02-16 17:55:00,220,NaN,NaN,2024,2,16,17.0,2024,2,16,18.0
3,dd0fad3248951d2f71d63e6279aeaa4b,XA-VBW,AW,AS,MX Amigos y Familia,Ciudad Principal,2023-06-26 15:15:00,2023-06-26 15:55:00,220,200.0,142.0,2023,6,26,15.0,2023,6,26,16.0
4,d0987ee648eea254063bfe2b39571b67,XA-VAP,BA,AB,Playa,Ciudad Principal,2023-02-10 08:40:00,2023-02-10 09:50:00,186,162.0,90.0,2023,2,10,9.0,2023,2,10,10.0


A high season column will be added, where 1 is high season and 0 is low season. This is because flights are affected by whether it is high or low season. It is determined if it is high season with the Mexican school calendar.

In [11]:
# Function to determine if the date is high season
def is_high_season(date):
    year = date.year
    if (pd.Timestamp(year=year, month=7, day=17) <= date <= pd.Timestamp(year=year, month=8, day=27)) or \
       (pd.Timestamp(year=year, month=11, day=18) <= date <= pd.Timestamp(year=year, month=11, day=20)) or \
       (pd.Timestamp(year=year, month=12, day=18) <= date <= pd.Timestamp(year=year+1, month=1, day=5)) or \
       (pd.Timestamp(year=year, month=2, day=3) <= date <= pd.Timestamp(year=year, month=2, day=5)) or \
       (pd.Timestamp(year=year, month=3, day=16) <= date <= pd.Timestamp(year=year, month=3, day=18)):
        return 1
    if year == 2024 and (pd.Timestamp(year=2024, month=3, day=25) <= date <= pd.Timestamp(year=2024, month=4, day=7)):
        return 1
    if year == 2023 and (pd.Timestamp(year=2023, month=4, day=1) <= date <= pd.Timestamp(year=2023, month=4, day=16)):
        return 1
    return 0

# Apply the function to the STD column to create a new column 'high_Season'
df_flights['high_season'] = df_flights['STD'].apply(is_high_season)

df_flights

,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,Bookings,STD_Year,STD_Month,STD_Day,STD_Hour,STA_Year,STA_Month,STA_Day,STA_Hour,high_season
0,ab954014077430bd842cfa305a55c0f8,XA-VBY,AT,AZ,Ciudad Fronteriza,Ciudad Principal,2023-10-19 11:40:00,2023-10-19 14:25:00,240,229.0,157.0,2023,10,19,12.0,2023,10,19,14.0,0
1,efd86c996035dacdca7a0ccb2560dda1,XA-VIX,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,2023-07-03 00:55:00,2023-07-03 04:55:00,186,197.0,109.0,2023,7,3,1.0,2023,7,3,5.0,0
2,6cfa1bbaa44f08fc7d3061f034a6a5ce,XA-VBV,AW,AS,MX Amigos y Familia,Ciudad Principal,2024-02-16 17:10:00,2024-02-16 17:55:00,220,NaN,NaN,2024,2,16,17.0,2024,2,16,18.0,0
3,dd0fad3248951d2f71d63e6279aeaa4b,XA-VBW,AW,AS,MX Amigos y Familia,Ciudad Principal,2023-06-26 15:15:00,2023-06-26 15:55:00,220,200.0,142.0,2023,6,26,15.0,2023,6,26,16.0,0
4,d0987ee648eea254063bfe2b39571b67,XA-VAP,BA,AB,Playa,Ciudad Principal,2023-02-10 08:40:00,2023-02-10 09:50:00,186,162.0,90.0,2023,2,10,9.0,2023,2,10,10.0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
245748,cc7c1c5e6fd132fd0bdab3a35aac33c0,XA-VBK,BM,BT,Playa,Ciudad Fronteriza,2023-12-29 07:30:00,2023-12-29 14:50:00,240,189.0,86.0,2023,12,29,7.0,2023,12,29,15.0,1
245749,ef32da2731db80faa8b9f5030979a016,9H-MLV,AW,BT,Playa,Ciudad Principal,2024-04-20 10:45:00,2024-04-20 14:00:00,178,NaN,NaN,2024,4,20,11.0,2024,4,20,14.0,0
245750,9c8970e9836d9c5ef9415bfa93c3f408,XA-VAC,AT,BT,Playa,Ciudad Principal,2023-12-22 12:50:00,2023-12-22 16:10:00,180,163.0,95.0,2023,12,22,13.0,2023,12,22,16.0,1
245751,3c15779202b13158f78e8a8afe377250,NaN,AW,BT,Playa,Ciudad Principal,2024-10-04 10:00:00,2024-10-04 13:15:00,240,NaN,NaN,2024,10,4,10.0,2024,10,4,13.0,0


A column will also be added that will indicate if it is a weekend, 1 if it is a weekend and 0 if it is not.

In [12]:
df_flights['Is_Weekend'] = df_flights['STD'].dt.dayofweek.apply(lambda x: 1 if x >= 5 else 0)

df_flights.head()

,Flight_ID,Aeronave,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,STD,STA,Capacity,Passengers,...,STD_Year,STD_Month,STD_Day,STD_Hour,STA_Year,STA_Month,STA_Day,STA_Hour,high_season,Is_Weekend
0,ab954014077430bd842cfa305a55c0f8,XA-VBY,AT,AZ,Ciudad Fronteriza,Ciudad Principal,2023-10-19 11:40:00,2023-10-19 14:25:00,240,229.0,...,2023,10,19,12.0,2023,10,19,14.0,0,0
1,efd86c996035dacdca7a0ccb2560dda1,XA-VIX,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,2023-07-03 00:55:00,2023-07-03 04:55:00,186,197.0,...,2023,7,3,1.0,2023,7,3,5.0,0,0
2,6cfa1bbaa44f08fc7d3061f034a6a5ce,XA-VBV,AW,AS,MX Amigos y Familia,Ciudad Principal,2024-02-16 17:10:00,2024-02-16 17:55:00,220,NaN,...,2024,2,16,17.0,2024,2,16,18.0,0,0
3,dd0fad3248951d2f71d63e6279aeaa4b,XA-VBW,AW,AS,MX Amigos y Familia,Ciudad Principal,2023-06-26 15:15:00,2023-06-26 15:55:00,220,200.0,...,2023,6,26,15.0,2023,6,26,16.0,0,0
4,d0987ee648eea254063bfe2b39571b67,XA-VAP,BA,AB,Playa,Ciudad Principal,2023-02-10 08:40:00,2023-02-10 09:50:00,186,162.0,...,2023,2,10,9.0,2023,2,10,10.0,0,0


## Predictive model

Only data from 2023 is going to be used to train and test the model since there are the only flights that contain the number of passengers

In [13]:
df_flights2023 = df_flights[df_flights['STD_Year'] == 2023]

The model we are selecting is random forest, we tried different models like XGBoost, linear regression, even a combination of the three models, but random forest was the one that perfomed better. The hyperparameters of the model were selected using a grid search.

For the categoric columns we tried using One Hot Encoding, but the dimensionality created was too large so we decided to use Target Encoder, which perfomed better.

In [14]:
X = df_flights2023.drop(columns = ['Passengers', 'Flight_ID','Aeronave', 'STD', 'STA', 'Bookings', 'STD_Year', 'STD_Day' ] , axis=1)
y = df_flights2023['Passengers']


In [15]:
X

,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,Capacity,STD_Month,STD_Hour,STA_Year,STA_Month,STA_Day,STA_Hour,high_season,Is_Weekend
0,AT,AZ,Ciudad Fronteriza,Ciudad Principal,240,10,12.0,2023,10,19,14.0,0,0
1,BM,AV,MX Amigos y Familia,Ciudad Fronteriza,186,7,1.0,2023,7,3,5.0,0,0
3,AW,AS,MX Amigos y Familia,Ciudad Principal,220,6,15.0,2023,6,26,16.0,0,0
4,BA,AB,Playa,Ciudad Principal,186,2,9.0,2023,2,10,10.0,0,0
5,AJ,AR,Playa,MX Amigos y Familia,240,9,17.0,2023,9,7,18.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
245718,AO,BT,Playa,Ciudad Principal,180,12,6.0,2023,12,26,10.0,1,0
245745,AW,BT,Playa,Ciudad Principal,180,12,10.0,2023,12,28,14.0,1,0
245747,BA,BT,Playa,Ciudad Principal,180,12,12.0,2023,12,23,15.0,1,1
245748,BM,BT,Playa,Ciudad Fronteriza,240,12,7.0,2023,12,29,15.0,1,0


In [16]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

# Create the encoder by specifying the categorical columns
encoder = ce.TargetEncoder(cols=['DepartureStation', 'ArrivalStation', 'Destination_Type', 'Origin_Type'])

# Tune the encoder with training data
encoder.fit(X_train, y_train)

# Transform the data
X_train_encoded = encoder.transform(X_train)
X_test_encoded = encoder.transform(X_test)


In [18]:
X_train

,DepartureStation,ArrivalStation,Destination_Type,Origin_Type,Capacity,STD_Month,STD_Hour,STA_Year,STA_Month,STA_Day,STA_Hour,high_season,Is_Weekend
112840,AI,AT,Ciudad Principal,Playa,186,8,14.0,2023,8,15,15.0,1,0
29060,AW,AK,Playa,Ciudad Principal,180,8,10.0,2023,8,13,13.0,1,1
95716,AP,AT,Ciudad Principal,MX Amigos y Familia,240,11,9.0,2023,11,29,12.0,0,0
109883,AZ,AT,Ciudad Principal,Ciudad Fronteriza,220,2,17.0,2023,2,7,19.0,0,0
67574,AW,AO,Ciudad Principal,Ciudad Principal,180,1,17.0,2023,1,2,18.0,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...
240068,AW,BP,MX Amigos y Familia,Ciudad Principal,240,2,6.0,2023,2,5,8.0,0,1
208102,AT,BJ,MX Amigos y Familia,Ciudad Principal,186,6,14.0,2023,6,4,16.0,0,1
1586,AW,AV,MX Amigos y Familia,Ciudad Principal,180,3,20.0,2023,3,24,22.0,0,0
30697,AL,AK,Playa,MX Amigos y Familia,220,9,8.0,2023,9,16,12.0,0,1


In [18]:
n_estimators = 300 
max_depth = 30     
min_samples_split = 5

rf_model  = RandomForestRegressor(n_estimators=n_estimators, max_depth=max_depth, min_samples_split = min_samples_split)

rf_model.fit(X_train_encoded, y_train)

# Make predictions on the test set
y_pred = rf_model.predict(X_test_encoded)

# Calculate and display R^2
r2 = r2_score(y_test, y_pred)
print(f"Determination Coefficient (R^2): {r2}")

Determination Coefficient (R^2): 0.6692267912514142


The coefficient of determination, often denoted as R-squared (R2), is a statistical measure that represents the proportion of the variance in the dependent variable that is predictable from the independent variables in a regression model. In simpler terms, R-squared indicates the goodness of fit of a regression model.

An R2 value of 0.66 means that approximately 66% of the variability in the dependent variable can be explained by the independent variables included in the model. In other words, the model accounts for 66% of the variability observed in the data. This suggests that the regression model is moderately effective in capturing the relationship between the independent and dependent variables. However, it also implies that about 34% of the variability remains unexplained by the model.

Since we need another metric to complement R2 we are going to use MAPE. The Mean Absolute Percentage Error (MAPE) is a metric used to evaluate the accuracy of a forecasting or predictive model. It measures the average absolute percentage difference between actual and predicted values, expressed as a percentage of the actual values.

A lower MAPE indicates higher accuracy, with 0% indicating a perfect prediction.

We complement R-squared (R2) with MAPE because R2 measures the goodness of fit of a regression model, capturing how well the model explains the variability in the data. However, R2 alone may not fully reflect the accuracy of the predictions, especially in cases where the magnitude of errors varies across observations. MAPE, on the other hand, directly measures the accuracy of predictions in terms of percentage error, providing additional insights into the performance of the model, especially in forecasting scenarios. By calculating both R2 and MAPE, we can gain a more comprehensive understanding of the model's effectiveness in capturing the relationship between variables and making accurate predictions.

In [19]:
def mean_absolute_percentage_error(y_true, y_pred): 
    """ Calcula el Mean Absolute Percentage Error entre y_true y y_pred """
    y_true, y_pred = np.array(y_true), np.array(y_pred)
    non_zero_mask = y_true != 0
    if not np.any(non_zero_mask):
        return np.inf  # Evita la división por cero si y_true es cero
    return np.mean(np.abs((y_true[non_zero_mask] - y_pred[non_zero_mask]) / y_true[non_zero_mask])) * 100

In [20]:
mape = mean_absolute_percentage_error(y_test, y_pred)
print(f"MAPE: {mape:.2f}%")

MAPE: 14.69%



A Mean Absolute Percentage Error (MAPE) of 14.69% indicates that, on average, the predictions made by the model have an absolute percentage error of 14.69% relative to the actual values. In other words, the model's predictions deviate from the actual values by approximately 14.69% on average.

For example, if the actual value of a variable is 100, a MAPE of 14.69% means that, on average, the model's predictions could be expected to deviate from this actual value by around 14.69 units.